In [196]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from pymystem3 import Mystem

RANDOM_SEED = 42
from sklearn.metrics import roc_auc_score

In [197]:
df_train = pd.read_csv('rusentitweet_train.csv')
df_test = pd.read_csv('rusentitweet_test.csv')

#оставляем только label = positive/negative
df_train = df_train[df_train['label'].isin(['positive', 'negative'])]
df_test = df_test[df_test['label'].isin(['positive', 'negative'])]
df_train

,text,label,id
0,Помойму я вкрашилась в Чимина🤧 https://t.co/t2...,positive,1282311169534038016
5,@buybread_ я не с порядке!!!!,negative,1335130757044563971
10,@ange1flyhigh В следующий раз буду до победног...,positive,1215370396465291267
15,@LimitaVIP Удивительный гiмн...\r\nУдивительно...,negative,1253799540848762887
17,я срала на эту биологию,negative,1339418979887173632
...,...,...,...
10704,"Дедрадио5 то есть ты хочешь сказать, что ты ла...",negative,1315037255833092098
10705,"@kmoo_m ДЕЙСТВИТЕЛЬНО\r\nесли любовь, то тольк...",positive,1323606772578459648
10708,С Хоби утро может быть только добрым😂 https://...,positive,1310485706213666816
10711,"Не успела встать, а уже заебалась, впрочем нич...",negative,1343031810746425344


In [198]:
russian_stopwords = stopwords.words("russian")

def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)   # Удаление ссылок
    tweet = re.sub(r'@\w+', '', tweet)      # Удаление упоминаний пользователей
    tweet = re.sub(r'#\w+', '', tweet)      # Удаление хештегов
    tweet = re.sub(r'\d+', '', tweet)       # Удаление чисел
    tweet = re.sub(r'[^\w\s]', '', tweet)   # Удаление пунктуации
    tweet = re.sub(r'[a-zA-Z]+', '', tweet) # Удаление слов на латинице
    tweet = re.sub(r'[^\w\s,]', '', tweet)  # Удаление эмодзи
    tweet = tweet.lower()                   # Приведение к нижнему регистру
    tweet = " ".join([word for word in tweet.split() if word not in russian_stopwords]) # Удаление стоп-слов
    tweet = re.sub(r'\s+', ' ', tweet).strip() # Удаление лишних пробелов
    tweet = re.sub(r'\b\w\b', '', tweet)   # Удаление слов из одной буквы
    return tweet

# Применение функции очистки к каждому твиту в наборах данных
df_train['text'] = df_train['text'].apply(clean_tweet)
df_test['text'] = df_test['text'].apply(clean_tweet)

# Применение функции очистки к каждому твиту в наборах данных
df_train['text'] = df_train['text'].apply(clean_tweet)
df_test['text'] = df_test['text'].apply(clean_tweet)

# Удаление пустых твитов
df_train = df_train[df_train['text'].str.strip().astype(bool)]
df_test = df_test[df_test['text'].str.strip().astype(bool)]

In [199]:
# Инициализируем стеммер для русского языка
stemmer = SnowballStemmer("russian")

def stem_tweet(tweet):
    # Разделяем твит на слова
    words = tweet.split()
    # Применяем стемминг к каждому слову
    stemmed_words = [stemmer.stem(word) for word in words]
    # Собираем обратно в строку
    return ' '.join(stemmed_words)

df_train_stem = df_train.copy()
df_test_stem = df_test.copy()

# Применяем функцию стемминга к каждому твиту
df_train_stem['text'] = df_train_stem['text'].apply(stem_tweet)
df_test_stem['text'] = df_test_stem['text'].apply(stem_tweet)

df_train_stem

,text,label,id
0,помойм вкраш чимин,positive,1282311169534038016
5,порядк,negative,1335130757044563971
10,след буд победн закрыва пожела удач,positive,1215370396465291267
15,удивительн гмн удивительн пок ещ сдохл украин,negative,1253799540848762887
17,срал биолог,negative,1339418979887173632
...,...,...,...
10704,дедрад хочеш сказа лайка мо твит пздц удаля отсюд,negative,1315037255833092098
10705,действительн любов так,positive,1323606772578459648
10708,хоб утр добр,positive,1310485706213666816
10711,успел встат заеба нов,negative,1343031810746425344


In [200]:
# Инициализация CountVectorizer
vectorizer = CountVectorizer()

# Обучение на тренировочных данных и преобразование их в мешок слов
X_train = vectorizer.fit_transform(df_train_stem['text'])

# Преобразование тестовых данных в мешок слов
X_test = vectorizer.transform(df_test_stem['text'])

In [201]:
# Создание объекта tf-idf transformer
tfidf_transformer = TfidfTransformer()

# Обучение на тренировочных данных и преобразование их
X_train_tfidf = tfidf_transformer.fit_transform(X_train)

# Преобразование тестовых данных
X_test_tfidf = tfidf_transformer.transform(X_test)

In [202]:
# Создание модели логистической регрессии
logreg = LogisticRegression(random_state=RANDOM_SEED)
logreg.fit(X_train_tfidf, df_train['label'])

# Создание модели случайного леса
rf = RandomForestClassifier(random_state=RANDOM_SEED)
rf.fit(X_train_tfidf, df_train['label'])

# Предсказание на тестовых данных
logreg_pred = logreg.predict(X_test_tfidf)
rf_pred = rf.predict(X_test_tfidf)

# Оценка результатов
print("Логистическая регрессия:\n", classification_report(df_test['label'], logreg_pred))
print("Случайный лес:\n", classification_report(df_test['label'], rf_pred))



# Convert labels in df_test_lem to numeric
numeric_labels = [0 if label == 'negative' else 1 for label in df_test_stem['label']]

# Convert predictions in logreg_pred_lem to numeric
numeric_predictions = [0 if pred == 'negative' else 1 for pred in logreg_pred]

roc_auc = roc_auc_score(numeric_labels, numeric_predictions)
print("ROC_AUC_SCORE логистическая регрессия c эмодзи:\n", roc_auc)
logreg_pred.shape()

Логистическая регрессия:
               precision    recall  f1-score   support

    negative       0.73      0.93      0.82       656
    positive       0.84      0.54      0.66       477

    accuracy                           0.76      1133
   macro avg       0.79      0.73      0.74      1133
weighted avg       0.78      0.76      0.75      1133

Случайный лес:
               precision    recall  f1-score   support

    negative       0.73      0.88      0.80       656
    positive       0.77      0.55      0.64       477

    accuracy                           0.74      1133
   macro avg       0.75      0.72      0.72      1133
weighted avg       0.75      0.74      0.73      1133

ROC_AUC_SCORE логистическая регрессия c эмодзи:
 0.7320444725673672


In [203]:
# Для логистической регрессии
feature_names = vectorizer.get_feature_names_out()
logreg_coef = logreg.coef_[0]
sorted_features = sorted(zip(logreg_coef, feature_names), reverse=True)
print("Наиболее важные положительные слова для логистической регрессии:")
sorted_words = [word for coef, word in sorted_features]
sorted_words_string = ' '.join(sorted_words[:10])
print(sorted_words_string)

sorted_features = sorted(zip(logreg_coef, feature_names))
print("Наиболее важные отрицательные слова для логистической регрессии:")
sorted_words = [word for coef, word in sorted_features]
sorted_words_string = ' '.join(sorted_words[:10])
print(sorted_words_string)

# Для случайного леса
rf_importances = rf.feature_importances_
# Получаем индексы наиболее важных признаков в порядке убывания их важности
sorted_indices = np.argsort(rf_importances)[::-1]
# Извлекаем соответствующие слова
top_words = [feature_names[i] for i in sorted_indices[:20]]
# Преобразуем список слов в строку, разделяя слова пробелами
top_words_string = ' '.join(top_words)
print("Наиболее важные слова для случайного леса:")
print(top_words_string)

Наиболее важные положительные слова для логистической регрессии:
любл красив прекрасн лучш мил классн крут нрав ва рад
Наиболее важные отрицательные слова для логистической регрессии:
блят пиздец нах сук хуйн вообщ заеба ненавиж грустн ужасн
Наиболее важные слова для случайного леса:
любл красив блят прекрасн крут лучш мил эт классн пиздец нрав хорош рад ва хоч сук очен любим обожа нах


In [204]:
# Инициализируем Mystem
mystem = Mystem()

def lemmatize_text(text):
    # Применяем лемматизацию к тексту и объединяем обратно в строку
    lemmas = mystem.lemmatize(text)
    return ''.join(lemmas).strip()

df_train_lem = df_train.copy()
df_test_lem = df_test.copy()

# Применяем функцию лемматизации к столбцу с текстом в DataFrame
df_train_lem['text'] = df_train_lem['text'].apply(lemmatize_text)
df_test_lem['text'] = df_test_lem['text'].apply(lemmatize_text)

In [205]:
# Инициализация CountVectorizer
vectorizer = CountVectorizer()

# Обучение на тренировочных данных и преобразование их в мешок слов
X_train_lem = vectorizer.fit_transform(df_train_lem['text'])

# Преобразование тестовых данных в мешок слов
X_test_lem = vectorizer.transform(df_test_lem['text'])

In [206]:
# Создание объекта tf-idf transformer
tfidf_transformer = TfidfTransformer()

# Обучение на тренировочных данных и преобразование их
X_train_tfidf_lem = tfidf_transformer.fit_transform(X_train_lem)

# Преобразование тестовых данных
X_test_tfidf_lem = tfidf_transformer.transform(X_test_lem)

In [207]:
# Создание модели логистической регрессии
logreg = LogisticRegression(random_state=RANDOM_SEED)
logreg.fit(X_train_tfidf_lem, df_train_lem['label'])

# Создание модели случайного леса
rf = RandomForestClassifier(random_state=RANDOM_SEED)
rf.fit(X_train_tfidf_lem, df_train_lem['label'])

# Предсказание на тестовых данных
logreg_pred_lem = logreg.predict(X_test_tfidf_lem)
rf_pred_lem = rf.predict(X_test_tfidf_lem)

# Оценка результатов
print("Логистическая регрессия:\n", classification_report(df_test_lem['label'], logreg_pred_lem))
print("Случайный лес:\n", classification_report(df_test_lem['label'], rf_pred_lem))

# Convert labels in df_test_lem to numeric
numeric_labels = [0 if label == 'negative' else 1 for label in df_test_lem['label']]

# Convert predictions in logreg_pred_lem to numeric
numeric_predictions = [0 if pred == 'negative' else 1 for pred in logreg_pred_lem]

roc_auc = roc_auc_score(numeric_labels, numeric_predictions)
print("ROC_AUC_SCORE логистическая регрессия без эмодзи:\n", roc_auc)

Логистическая регрессия:
               precision    recall  f1-score   support

    negative       0.74      0.92      0.82       656
    positive       0.84      0.56      0.67       477

    accuracy                           0.77      1133
   macro avg       0.79      0.74      0.75      1133
weighted avg       0.79      0.77      0.76      1133

Случайный лес:
               precision    recall  f1-score   support

    negative       0.73      0.87      0.79       656
    positive       0.76      0.55      0.63       477

    accuracy                           0.74      1133
   macro avg       0.74      0.71      0.71      1133
weighted avg       0.74      0.74      0.73      1133

ROC_AUC_SCORE логистическая регрессия без эмодзи:
 0.7428126757682671


In [208]:
# Для логистической регрессии
feature_names = vectorizer.get_feature_names_out()
logreg_coef = logreg.coef_[0]
sorted_features = sorted(zip(logreg_coef, feature_names), reverse=True)
print("Наиболее важные положительные слова для логистической регрессии:")
sorted_words = [word for coef, word in sorted_features]
sorted_words_string = ' '.join(sorted_words[:10])
print(sorted_words_string)

sorted_features = sorted(zip(logreg_coef, feature_names))
print("Наиболее важные отрицательные слова для логистической регрессии:")
sorted_words = [word for coef, word in sorted_features]
sorted_words_string = ' '.join(sorted_words[:10])
print(sorted_words_string)

# Для случайного леса
rf_importances = rf.feature_importances_
# Получаем индексы наиболее важных признаков в порядке убывания их важности
sorted_indices = np.argsort(rf_importances)[::-1]
# Извлекаем соответствующие слова
top_words = [feature_names[i] for i in sorted_indices[:20]]
# Преобразуем список слов в строку, разделяя слова пробелами
top_words_string = ' '.join(top_words)
print("Наиболее важные слова для случайного леса:")
print(top_words_string)

Наиболее важные положительные слова для логистической регрессии:
любить хороший красивый милый прекрасный классный вау нравиться крутой любовь
Наиболее важные отрицательные слова для логистической регрессии:
блять пиздец нахуй сука умирать вообще хуйня ненавидеть уходить сдыхать
Наиболее важные слова для случайного леса:
любить хороший блять красивый это пиздец нравиться вау прекрасный хотеть любовь милый классный крутой самый очень вообще красиво умирать рад
